In [1]:
# If needed 
# !pip install praat-textgrids



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
import pandas as pd
import textgrids
import mne
import mne_bids
import numpy as np

In [5]:
# For each of the 9 run:

for run in np.arange(1,10):

    # Open the text grid
    grid = textgrids.TextGrid(f'./stimuli/grid/text_french_run{run}.TextGrid')

    dict_ts_to_phoneme = {} # Dict linking timestamp to phoneme

    # Get the list of phonemes, their timestamp, and create a dict based off it
    for ph in grid['MAU']:
        dict_ts_to_phoneme[ph.mid] = ph.text

    # Explore the TextGrid file, and add the voiced information
    # Now that all the text grids are generated, it is needed to add to each Text Grid the 
    # Following information: is it a voiced or a non-voiced phoneme?

    # Voiced phonemes are the following:
    voiced = ['Z','z','v','R','D','d','b']

    # All the other phonemecs are then non-voiced

    dict_ts_to_voiced = {}

    for key in dict_ts_to_phoneme:
        if dict_ts_to_phoneme[key] in voiced:
            dict_ts_to_voiced[key] = 'voiced'
        else:
            dict_ts_to_voiced[key] = 'non-voiced'

    # Deleting the first phoneme as it is an error:
    dict_ts_to_voiced.pop(list(dict_ts_to_voiced.keys())[0])


    # Now load the MEG recording, and add this information as annotations
    TASK = 'listen'

    subject = '1'

    # path_bids = '/home/co/workspace_LPP/data/MEG/LPP/LPP_bids'
    path_bids = '/home/is153802/data/BIDS_final/'


    bids_path = mne_bids.BIDSPath(
                subject=subject,
                session='01',
                task=TASK,
                datatype="meg",
                root=path_bids,
                run='0'+str(run),
            )

    # raw = mne_bids.read_raw_bids(bids_path)

    events_path = str(bids_path.fpath)[:-7] + 'events.tsv'

    events = pd.read_csv(events_path,delimiter = '\t')[['onset','duration','trial_type']]

    # Now we have to:

    # Align the phonemes timestamps with the current onset of words as in the events.tsv file

    # Add them as a kind: phoneme

    # There is an offset between the first word and the first phoneme: let's reajust

    offset = events.onset[0] - list(dict_ts_to_voiced.keys())[0]

    print(f'Offset is : {offset}')

    for timestamp in dict_ts_to_voiced.keys():
        timestamp_final = timestamp + offset 
        to_add = pd.DataFrame([timestamp_final, 0.02, {'kind':'phoneme','voice':dict_ts_to_voiced[timestamp]}])
        to_add_t = to_add.transpose()
        to_add_t.columns = events.columns
        events = pd.concat([events, to_add_t])
    
    events.to_csv(str(bids_path.fpath)[:-7] + 'events.tsv', sep='\t',index=False)



Offset is : 0.9228799999999997
Offset is : -0.012119999999999909
Offset is : 0.027880000000000127
Offset is : 0.0714399999999995
Offset is : 0.00788000000000011
Offset is : 0.017879999999999896
Offset is : 0.0028800000000002157
Offset is : 0.04287999999999981
Offset is : 0.06288000000000027


In [ ]:
for sub in np.arange(2,20):
    # For each of the 9 run:

    for run in np.arange(1,10):

        # Open the text grid
        grid = textgrids.TextGrid(f'./stimuli/grid/text_french_run{run}.TextGrid')

        dict_ts_to_phoneme = {} # Dict linking timestamp to phoneme

        # Get the list of phonemes, their timestamp, and create a dict based off it
        for ph in grid['MAU']:
            dict_ts_to_phoneme[ph.mid] = ph.text

        # Explore the TextGrid file, and add the voiced information
        # Now that all the text grids are generated, it is needed to add to each Text Grid the 
        # Following information: is it a voiced or a non-voiced phoneme?

        # Voiced phonemes are the following:
        voiced = ['Z','z','v','R','D','d','b']

        # All the other phonemecs are then non-voiced

        dict_ts_to_voiced = {}

        for key in dict_ts_to_phoneme:
            if dict_ts_to_phoneme[key] in voiced:
                dict_ts_to_voiced[key] = 'voiced'
            else:
                dict_ts_to_voiced[key] = 'non-voiced'

        # Deleting the first phoneme as it is an error:
        dict_ts_to_voiced.pop(list(dict_ts_to_voiced.keys())[0])


        # Now load the MEG recording, and add this information as annotations
        TASK = 'listen'

        subject = str(sub)

        # path_bids = '/home/co/workspace_LPP/data/MEG/LPP/LPP_bids'
        path_bids = '/home/is153802/data/BIDS_final/'


        bids_path = mne_bids.BIDSPath(
                    subject=subject,
                    session='01',
                    task=TASK,
                    datatype="meg",
                    root=path_bids,
                    run='0'+str(run),
                )

        # raw = mne_bids.read_raw_bids(bids_path)

        events_path = str(bids_path.fpath)[:-7] + 'events.tsv'

        events = pd.read_csv(events_path,delimiter = '\t')[['onset','duration','trial_type']]

        # Now we have to:

        # Align the phonemes timestamps with the current onset of words as in the events.tsv file

        # Add them as a kind: phoneme

        # There is an offset between the first word and the first phoneme: let's reajust

        offset = events.onset[0] - list(dict_ts_to_voiced.keys())[0]

        print(f'Offset is : {offset}')

        for timestamp in dict_ts_to_voiced.keys():
            timestamp_final = timestamp + offset 
            to_add = pd.DataFrame([timestamp_final, 0.02, {'kind':'phoneme','voice':dict_ts_to_voiced[timestamp]}])
            to_add_t = to_add.transpose()
            to_add_t.columns = events.columns
            events = pd.concat([events, to_add_t])
        
        events.to_csv(str(bids_path.fpath)[:-7] + 'events.tsv', sep='\t',index=False)

